# Species wrangling
In this notebook, I'll combine some information about which species we're interested in using for this project (from manually created spreadsheets) and find sources for genomes/transcriptomes for those we don't already have.

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, exists
from openpyxl import load_workbook
from collections import defaultdict
import numpy as np
from pysradb.sraweb import SRAweb
from tqdm import tqdm
import subprocess

/mnt/home/lotrecks/.conda/envs/omics/lib/python3.10/site-packages/pysradb/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
run_date = '16Sep2025'

## Reading in the data

In [3]:
phyto_origin = pd.read_csv('../data/species_info/species_info_full.csv')
phyto_origin.head()

,abbreviated_species,species_gbif,species,common_name,has_genome,has_transcriptome,original_source,thermotolerance,reference,notes
0,A. americanus,Acorus americanus,Acorus americanus,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
1,A. tequilanavar,Agave tequilana,Agave tequilana,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
2,A. officinalis,Althaea officinalis,Althaea officinalis,marsh-mallow,NaN,NaN,phytozome,NaN,NaN,NaN
3,A. linifolium,Alyssum linifolium,Alyssum linifolium,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
4,A. hypochondriacus,Amaranthus hypochondriacus,Amaranthus hypochondriacus,NaN,NaN,NaN,phytozome,NaN,NaN,NaN


Add a path for the genomes we already have:

In [4]:
genome_top_path = '/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/phytozome_genomes'
sp_path = [(f"{f.split('_')[0][0]}. {f.split('_')[0][1:]}", f'{genome_top_path}/{f}') for f in listdir(genome_top_path)]
sp_path_df = pd.DataFrame(sp_path, columns=['abbreviated_species', 'full_path'])
phyto_paths = sp_path_df.merge(phyto_origin, left_on='abbreviated_species', right_on='abbreviated_species', how='right')

Include function to read URLs from Excel files:

In [5]:
def get_hyperlink_columns(base_df, path, sheet_name):
    """
    Use openpyxl to detect hyperlinks and add them as new columns.

    From: https://github.com/serenalotreck/greencut-formatting/blob/main/notebooks/greencut_database_input_formatting.ipynb

    parameters:
        base_df, pandas df: the result of read_excel on path
        path, str: path to the excel document
        sheet_name, str: name of the sheet to read

    returns:
        updated_df, pandas df: base_df with added columns for hyperlinks
    """
    # Load the workbook using openpyxl
    workbook = load_workbook(path)
    
    # Select the sheet
    sheet = workbook[sheet_name] if sheet_name else workbook.active

    # Openpyxl will maintain rows that aren't relevant to the dataframe,
    # whereas pandas.read_excel just drops those -- we need to make them agree
    if sheet.max_row != len(base_df):
        rows_to_delete = [i for i in range(len(base_df) + 1, sheet.max_row)]
        for row_index in reversed(rows_to_delete):
            sheet.delete_rows(row_index)

    # Detect columns with hyperlinks
    hyperlink_cols = defaultdict(list)
    for row_idx, row in enumerate(sheet.iter_rows()):
        # Skip the first row, since it's the header
        if row_idx == 0:
            continue
        for col_index, cell in enumerate(row):
            hyperlink = cell.hyperlink.target if cell.hyperlink else None
            header_name = sheet.cell(row=1, column=col_index + 1).value
            if hyperlink:
                hyperlink_cols[header_name + "_hyperlink"].append(hyperlink)
            else:
                # We need to add NaN if there's no hyperlink to keep the values
                # in the right rows, but this does mean we'll get non-hyperlink columns
                hyperlink_cols[header_name + "_hyperlink"].append(np.nan)

    # Create a df with the new columns
    hyperlink_df = pd.DataFrame(hyperlink_cols)
    
    # Now remove any columns that are all NaN
    hyperlink_df = hyperlink_df.dropna(axis=1, how='all')

    # Concat this horizontally to the original df
    updated_df = pd.concat([base_df, hyperlink_df], axis=1)

    return updated_df

In [6]:
with_lit = pd.read_excel('../data/species_info/thermophilic_plant_species.xlsx', sheet_name='Sheet1')
with_lit.columns = with_lit.columns.str.replace(' ', '_')
with_list = with_lit.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
with_lit.head()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes
0,Heliotropium curisavvicum,a prostrate plant from the Mojave desert. You...,NaN,NaN,Rowan suggested
1,Sesuvium verrucosum,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested
2,Sesuvium trianthema,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested
3,Ipomea pes-caprae,"tropical beach vine, grows on hot sands of tro...",NaN,NaN,Rowan suggested
4,Mollugo verticillata,carpet weed. Grows across blacktop that burns...,NaN,NaN,Rowan suggested


## Wrangling
Drop specific rows we know don't have info from manually looking:

In [7]:
with_lit = with_lit.drop(index=[60,125])
with_lit.tail()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes
120,Azorella trifurcata (Gaertn.) Pers.,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
121,Azorella trisecta (H.Wolff) Mart.Fernández & C...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
122,Azorella ulicina (Gillies & Hook.) G.M.Plunket...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
123,Azorella valentini (Speg.) Mart.Fernández & C....,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
124,Cistanthe longiscapa,CAM species that grows in the atacama desert,NaN,NaN,"Paper from Ariel Orellana, gave a talk in the ..."


In [8]:
lit_w_links = get_hyperlink_columns(with_lit, '../data/species_info/thermophilic_plant_species.xlsx', sheet_name='Sheet1').drop(index=60)
lit_w_links.tail()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes,Species Name_hyperlink
120,Azorella trifurcata (Gaertn.) Pers.,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
121,Azorella trisecta (H.Wolff) Mart.Fernández & C...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
122,Azorella ulicina (Gillies & Hook.) G.M.Plunket...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
123,Azorella valentini (Speg.) Mart.Fernández & C....,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",NaN
124,Cistanthe longiscapa,CAM species that grows in the atacama desert,NaN,NaN,"Paper from Ariel Orellana, gave a talk in the ...",NaN


Get rid of additional text after the first two space-delimited words in the species column to get a base species name.

In [9]:
lit_w_links['species'] = lit_w_links['Species_Name'].str.split(r'[\xa0\s]+').str[:2].str.join(' ')

In [10]:
lit_w_links = lit_w_links.rename(columns={'Notes': 'original_source', 'Species_Notes': 'notes', 'Publication_Link': 'reference'})
lit_w_links.head()

,Species_Name,notes,Transciptomic_data_in_NCBI_with_SRA_data,reference,original_source,Species Name_hyperlink,species
0,Heliotropium curisavvicum,a prostrate plant from the Mojave desert. You...,NaN,NaN,Rowan suggested,NaN,Heliotropium curisavvicum
1,Sesuvium verrucosum,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested,NaN,Sesuvium verrucosum
2,Sesuvium trianthema,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested,NaN,Sesuvium trianthema
3,Ipomea pes-caprae,"tropical beach vine, grows on hot sands of tro...",NaN,NaN,Rowan suggested,NaN,Ipomea pes-caprae
4,Mollugo verticillata,carpet weed. Grows across blacktop that burns...,NaN,NaN,Rowan suggested,NaN,Mollugo verticillata


Merge with the other list:

In [11]:
phyto_paths['reference'] = phyto_paths['reference'].astype('object')
all_species = lit_w_links.merge(phyto_paths, left_on=['species', 'notes', 'reference'], right_on=['species', 'notes', 'reference'], suffixes=('_berkley', '_serena'), how='outer')
all_species = all_species.drop(columns=['Species_Name', 'Transciptomic_data_in_NCBI_with_SRA_data', 'Species Name_hyperlink', 'abbreviated_species'])
all_species = all_species[['species', 'thermotolerance'] + sorted(c for c in all_species.columns if c != 'species' and c != 'thermotolerance')]
all_species = all_species.drop_duplicates()
all_species.head()

,species,thermotolerance,common_name,full_path,has_genome,has_transcriptome,notes,original_source_berkley,original_source_serena,reference,species_gbif
0,Acorus americanus,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Acorus americanus
1,Agave sp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agave tequilana,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Agave tequilana
3,Althaea officinalis,NaN,marsh-mallow,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Althaea officinalis
4,Alyssum linifolium,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Alyssum linifolium


How many genera are represented here?

In [12]:
unique_genera = all_species.species.str.split(' ').str[0].value_counts()
print(f'There are {len(unique_genera)} unique genera in the list. The top genus is {unique_genera.idxmax()}, with {unique_genera.max()} species')
spec_in_common = set(phyto_paths.species).intersection(set(lit_w_links.species))
print(f'There were only {len(spec_in_common)} species in common between the two lists: {", ".join(list(spec_in_common))}')

There are 159 unique genera in the list. The top genus is Azorella, with 58 species
There were only 14 species in common between the two lists: Boechera crandallii, Boechera pallidifolia, Salvadora persica, Avicennia marina, Phoenix dactylifera, Boechera perennans, Boechera gunnisoniana, Boechera depauperata, Boechera stricta, Calligonum comosum, Haloxylon salicornicum, Salsola villosa, Boechera holboellii, Vigna unguiculata


Save the final thing:

In [13]:
all_species.to_csv(f'../data/species_info/berkley_serena_combined_species_list_{run_date}.csv', index=False)

## Looking in SRA
We want to know, before we manually search up all of these, that there's at least one entry in SRA for them. We'll use [`pysradb`](https://github.com/saketkc/pysradb) for this. We also can only use RNA-Seq data, so we'll also filter for that here.

In [15]:
db = SRAweb()

In [16]:
all_studies = {}
missing_rnaseq = []
for spec in tqdm(all_species.species):
    try:
        if isfile(all_species.loc[all_species.species==spec, 'full_path'].values[0]):
            print(f'Skipping species {spec} because it has a genome in Phytozome')
            continue
    except TypeError:
        studies = db.search_sra(spec)
        try:
            studies_rna = studies[studies.library_strategy.str.contains('RNA')]
            if len(studies_rna) > 0:
                print(f'There are {len(studies_rna)} RNA-seq studies in SRA for species {spec}')
                all_studies[spec] = studies_rna
            else:
                print(f'0 RNA-seq studies found for species {spec}.')
                missing_rnaseq.append(spec)
        except AttributeError:
            print(f'0 RNA-seq studies found for species {spec}.')
            missing_rnaseq.append(spec)

  0%|          | 0/294 [00:00<?, ?it/s]

Skipping species Acorus americanus because it has a genome in Phytozome


  1%|          | 2/294 [00:00<00:40,  7.16it/s]

0 RNA-seq studies found for species Agave sp..
Skipping species Agave tequilana  because it has a genome in Phytozome
Skipping species Althaea officinalis because it has a genome in Phytozome
Skipping species Alyssum linifolium because it has a genome in Phytozome
Skipping species Amaranthus hypochondriacus because it has a genome in Phytozome
Skipping species Amborella trichopoda because it has a genome in Phytozome
Skipping species Anacardium occidentale because it has a genome in Phytozome
Skipping species Ananas comosus because it has a genome in Phytozome
Skipping species Andropogon gerardi because it has a genome in Phytozome
Skipping species Aquilegia caerulea because it has a genome in Phytozome
Skipping species Arabidopsis halleri because it has a genome in Phytozome
Skipping species Arabidopsis lyrata because it has a genome in Phytozome
Skipping species Arabidopsis thaliana because it has a genome in Phytozome
Skipping species Arachis hypogaea because it has a genome in Phyt

  5%|▌         | 16/294 [00:03<00:56,  4.96it/s]

...

There are 58 RNA-seq studies in SRA for species Yucca brevifolia
Skipping species Zea mays because it has a genome in Phytozome


 99%|█████████▉| 292/294 [02:05<00:00,  4.06it/s]

There are 36 RNA-seq studies in SRA for species Ziziphus spina


100%|██████████| 294/294 [02:06<00:00,  2.32it/s]

There are 49 RNA-seq studies in SRA for species Ziziphus spina christi
Skipping species Zostera marina because it has a genome in Phytozome


In [17]:
print(f'There are {len(all_studies)} organisms with RNAseq datasets that also don\'t have Phytozome genomes, {len(list(listdir(genome_top_path)))} organisms with Phytozome genomes, and {len(missing_rnaseq)} that don\'t have an RNAseq dataset in SRA.')

There are 32 organisms with RNAseq datasets that also don't have Phytozome genomes, 162 organisms with Phytozome genomes, and 101 that don't have an RNAseq dataset in SRA.


In [19]:
print([len(stud) for stud in all_studies.values()], sum([len(stud) for stud in all_studies.values()]))

[127, 1, 6, 16, 124, 3, 12, 5, 3, 3, 124, 11, 3, 3, 12, 5, 18, 5, 16, 16, 1, 2, 6, 340, 157, 11, 2, 1, 1, 58, 36, 49] 1177


Concat so that we can just delete the studies we don't want from a single spreadsheet:

In [22]:
all_studies_df = pd.concat(all_studies.values()).drop_duplicates()
all_studies_df.head()

,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,library_source,...,biosample,bioproject,instrument,instrument_model,instrument_model_desc,total_spots,total_size,run_accession,run_total_spots,run_total_bases
738,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,82927,Avicennia marina,None,RNA-Seq,TRANSCRIPTOMIC,...,SAMD00108023,PRJDB6605,Illumina HiSeq 1500,Illumina HiSeq 1500,ILLUMINA,14681880,1398935741,DRR119672,14681880,2965739760
683,SRP233272,Tissue-specific transcriptomes from Avicennia ...,SRX7217273,Tissue-specific transcriptomes from Avicennia ...,Tissue-specific transcriptomes from Avicennia ...,82927,Avicennia marina,Transcriptomic reads of Avicennia_marina_Flowers,RNA-Seq,TRANSCRIPTOMIC,...,SAMN13391520,PRJNA591919,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,175350106,11323323651,SRR10533306,175350106,16828597954
621,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_Leaf_salt_0h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447932,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,11648210,757097881,SRR12145926,11648210,1750406459
620,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666716,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_24h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447933,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,11745346,763743587,SRR12145992,11745346,1765513323
619,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666717,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_48h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447934,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,13807034,893255733,SRR12145993,13807034,2074839242


Save out the studies so we don't have to run the search again:

In [23]:
all_studies_df.to_csv(f'/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/all_SRA_studies_{run_date}.csv', index=False)

Read them back in:

In [4]:
all_studies_df = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/all_SRA_studies_12Sep2025.csv')
all_studies_df.head()

,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,library_source,...,biosample,bioproject,instrument,instrument_model,instrument_model_desc,total_spots,total_size,run_accession,run_total_spots,run_total_bases
0,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,82927,Avicennia marina,NaN,RNA-Seq,TRANSCRIPTOMIC,...,SAMD00108023,PRJDB6605,Illumina HiSeq 1500,Illumina HiSeq 1500,ILLUMINA,14681880,1398935741,DRR119672,14681880,2965739760
1,SRP233272,Tissue-specific transcriptomes from Avicennia ...,SRX7217273,Tissue-specific transcriptomes from Avicennia ...,Tissue-specific transcriptomes from Avicennia ...,82927,Avicennia marina,Transcriptomic reads of Avicennia_marina_Flowers,RNA-Seq,TRANSCRIPTOMIC,...,SAMN13391520,PRJNA591919,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,175350106,11323323651,SRR10533306,175350106,16828597954
2,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_Leaf_salt_0h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447932,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,11648210,757097881,SRR12145926,11648210,1750406459
3,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666716,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_24h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447933,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,11745346,763743587,SRR12145992,11745346,1765513323
4,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666717,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_48h,RNA-Seq,TRANSCRIPTOMIC,...,SAMN15447934,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,13807034,893255733,SRR12145993,13807034,2074839242


We want to perform further filtering on this list to get a single rep per organism. This will require some if/else logic on the studies for each organism. First, we'll drop any `SINGLE` in the `library_layout` column, if there is a `PAIRED` option for that species, because paired end reads are more reliable.

In [12]:
drop_for_single = []
for species in all_studies_df.organism_name.unique():
    spec_subset = all_studies_df[all_studies_df.organism_name == species]
    single_end = spec_subset[spec_subset.library_layout == 'SINGLE']
    if len(single_end) == len(spec_subset): # Means there's only single end
        print(f'There are only single-end reads for species {species}')
        continue
    else:
        drop_for_single.extend(single_end.index.tolist())
        if len(single_end) > 0:
            print(f'For species {species}, {len(single_end)} of {len(spec_subset)} total studies are single-end reads, and will be dropped')

For species Avicennia marina, 5 of 88 total studies are single-end reads, and will be dropped
There are only single-end reads for species Boechera depauperata
For species Boechera divaricarpa, 5 of 16 total studies are single-end reads, and will be dropped
For species Boechera stricta, 17 of 41 total studies are single-end reads, and will be dropped
There are only single-end reads for species Arabidopsis thaliana
For species Boechera williamsii, 6 of 12 total studies are single-end reads, and will be dropped
For species Boechera pallidifolia, 8 of 11 total studies are single-end reads, and will be dropped
There are only single-end reads for species Opuntia robusta
There are only single-end reads for species Opuntia joconostle
There are only single-end reads for species Opuntia ficus-indica
For species Phoenix dactylifera, 23 of 267 total studies are single-end reads, and will be dropped
There are only single-end reads for species Coccotrypes dactyliperda
There are only single-end reads

In [13]:
all_studies_df = all_studies_df.drop(index=drop_for_single)

Unfortunately the read length is not included in the metadata returned by our search, so we can't filter based on that; we'll have to check that manually at the end. However, we also want to know that our samples are coming from leaf tissue. While there is not a eddicated field for tissue type, many studies inclued this information in one of the following fields: `study_title`, `experiment_title`, or `experiment_desc`. As for the single vs. paired filter, we'll look for any rows with the string `"leaf"` in them, but if none have them, we'll just keep what's there.

In [18]:
drop_for_leaf = []
for species in all_studies_df.organism_name.unique():
    spec_subset = all_studies_df[all_studies_df.organism_name == species]
    has_leaf = spec_subset[(spec_subset.study_title.str.lower().str.contains('leaf')) | (spec_subset.experiment_title.str.lower().str.contains('leaf')) | (spec_subset.experiment_desc.str.lower().str.contains('leaf'))]
    if len(has_leaf) == 0: # Means there's no leaf specific one
        print(f'There are no leaf-specific studies for species {species}')
        continue
    else:
        not_leaf_idxs = spec_subset.drop(index=has_leaf.index.tolist()).index.tolist()
        drop_for_leaf.extend(not_leaf_idxs)
        print(f'For species {species}, {len(has_leaf)} of {len(spec_subset)} total studies are leaf specific, all others will be dropped')

For species Avicennia marina, 17 of 83 total studies are leaf specific, all others will be dropped
There are no leaf-specific studies for species mangrove metagenome
For species Rhizophora apiculata, 6 of 12 total studies are leaf specific, all others will be dropped
For species Avicennia marina var. rumphiana, 2 of 2 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. eucalyptifolia, 1 of 2 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. australasica, 1 of 1 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. marina, 1 of 1 total studies are leaf specific, all others will be dropped
There are no leaf-specific studies for species sediment metagenome
There are no leaf-specific studies for species Azorella trisecta
There are no leaf-specific studies for species Boechera depauperata
There are no leaf-specific studies for species Boechera divaricarpa
There

In [19]:
all_studies_df = all_studies_df.drop(index=drop_for_leaf)

The last thing we can do tom ake this list easiest to manage is to remove duplicate entries from the same experiment, since we only need one rep. Any species that still have multiple experiments we will include one rep from all of them, as we can manually check the read lengths to choose among them. The groupings will have the same Bioproject number, but different sample accession numbers and different experiment numbers. We'll go through each remaining bioproject and keep just the first sample.

In [24]:
drop_for_reps = []
for proj in all_studies_df.bioproject.unique():
    bio_subset = all_studies_df[all_studies_df.bioproject == proj]
    all_but_first = bio_subset.index.tolist()[1:]
    drop_for_reps.extend(all_but_first)
print(f'{len(drop_for_reps)} replicate rows will be dropped (out of a total of {len(all_studies_df)})')

455 replicate rows will be dropped (out of a total of 512)


In [26]:
all_studies_df = all_studies_df.drop(index=drop_for_reps)

Save out what we have:

In [27]:
all_studies_df.to_csv(f'/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_{run_date}.csv', index=False)

## Getting fastq files
After exporting the above file, I manually checked read lengths and removed most duplicate samples. For those with similar length paired read samples, I left duplicates if (a) I thought it would be valuable or (b) if I wasn't certain about the tissue origin for the samples. Now, we'll use the SRA toolkit to prefetch and download the fastq files for each sample.

In [5]:
final_df = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_15Sep2025_manual_final_filtering.csv')
final_df.head()

,organism_name,len_reads,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,library_name,library_strategy,...,biosample,bioproject,instrument,instrument_model,instrument_model_desc,total_spots,total_size,run_accession,run_total_spots,run_total_bases
0,Adenophora stricta,150,SRP549143,This BioProject includes raw sequencing data f...,SRX26957374,leaf+stem RNA of Adenophora stricta,leaf+stem RNA of Adenophora stricta,82279,am_rnaseq_leaf,RNA-Seq,...,SAMN15447991,PRJNA392014,NextSeq 500,NextSeq 500,ILLUMINA,54006517,6346122539,SRR12146026,54006517,14987554557
1,Avicennia marina,150,SRP416093,The RNA-seq data of Avicennia marina leaf epid...,SRX18949618,RNA-Seq of A. marina,RNA-Seq of A. marina,82927,NaN,RNA-Seq,...,SAMN03654442,PRJNA283781,NextSeq 500,NextSeq 500,ILLUMINA,37876026,2883978460,SRR2029733,37876026,5757155952
2,Avicennia marina,90,SRP018275,Avicennia marina Transcriptome or Gene expression,SRX220549,Leaf transcriptome from Avicennia marina,Leaf transcriptome from Avicennia marina,82927,FKRO210225575-1A,RNA-Seq,...,SAMN32605810,PRJNA918785,Illumina NovaSeq 6000,Illumina NovaSeq 6000,ILLUMINA,133333541,12419487041,SRR22993527,133333541,40000062300
3,Avicennia marina var. rumphiana,150,SRP365449,Mangrove Genome Project,SRX14582339,Arumphiana_leaf,Arumphiana_leaf,1238153,AL04,RNA-Seq,...,SAMN41662877,PRJNA797928,BGISEQ-500,BGISEQ-500,BGISEQ,2365917,490760605,SRR29297605,2365917,695579598
4,Azorella trisecta,101,SRP299551,Plant ecological genomics at the limits of lif...,SRX9744420,RNA-seq of Azorella atacamensis,RNA-seq of Azorella atacamensis,1277843,NaN,RNA-Seq,...,SAMN01906725,PRJNA187540,Illumina Genome Analyzer IIx,Illumina Genome Analyzer IIx,ILLUMINA,20000000,2230394226,SRR653719,20000000,3600000000


The below code prefetches and downloads the data; however, the download step is prohibitively slow to run interactively. I'm going to submit a job to finish the download; the code is left here for reference.

In [8]:
# runs_to_download = final_df.run_accession.tolist()

# print('Prefetching...')
# for sra_id in tqdm(runs_to_download):
#     prefetch = "prefetch " + sra_id
#     subprocess.call(prefetch, shell=True)

# print('Downloading...')
# for sra_id in tqdm(runs_to_download):
#     fastq_dump = "fastq-dump --outdir /mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_downloads --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip /mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_downloads/sra_prefetch/sra/" + sra_id + ".sra"
#     subprocess.call(fastq_dump, shell=True)